In [ ]:
import os
os.environ["HOME"]

In [ ]:
import tensorflow as tf
print(tf.__version__)
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [ ]:
import numpy as np
def one_hot_encoder(s):
    pep_num = ' '.join([str(ints) for ints in range(20)])
    pep_ref = 'A C D E F G H I K L M N P Q R S T V W Y'
    d = dict(zip(pep_ref.split(' '), pep_num.split(' ')))
    
    x = np.zeros((len(d), len(s)))
    x[[int(d[c]) for c in s], range(len(s))] = 1
    return x

In [ ]:
import pandas as pd
folder = '/file_location/'
train_seq = pd.read_csv('{}/Seq_with_freq_train.csv'.format(folder), sep = "\t")
test_seq = pd.read_csv('{}/Seq_with_freq_test.csv'.format(folder), sep = "\t")

In [ ]:
#Translate the amino acid sequences to protein sequences
def translate(seq):
    table = {
        'ATA':'I', 'ATC':'I', 'ATT':'I', 'ATG':'M',
        'ACA':'T', 'ACC':'T', 'ACG':'T', 'ACT':'T',
        'AAC':'N', 'AAT':'N', 'AAA':'K', 'AAG':'K',
        'AGC':'S', 'AGT':'S', 'AGA':'R', 'AGG':'R',                 
        'CTA':'L', 'CTC':'L', 'CTG':'L', 'CTT':'L',
        'CCA':'P', 'CCC':'P', 'CCG':'P', 'CCT':'P',
        'CAC':'H', 'CAT':'H', 'CAA':'Q', 'CAG':'Q',
        'CGA':'R', 'CGC':'R', 'CGG':'R', 'CGT':'R',
        'GTA':'V', 'GTC':'V', 'GTG':'V', 'GTT':'V',
        'GCA':'A', 'GCC':'A', 'GCG':'A', 'GCT':'A',
        'GAC':'D', 'GAT':'D', 'GAA':'E', 'GAG':'E',
        'GGA':'G', 'GGC':'G', 'GGG':'G', 'GGT':'G',
        'TCA':'S', 'TCC':'S', 'TCG':'S', 'TCT':'S',
        'TTC':'F', 'TTT':'F', 'TTA':'L', 'TTG':'L',
        'TAC':'Y', 'TAT':'Y', 'TAA':'*', 'TAG':'*',
        'TGC':'C', 'TGT':'C', 'TGA':'*', 'TGG':'W'}
    j = [table.get(seq[i:i+3]) for i in range(0, len(seq), 3)]
    protein = "".join(j)
    return protein

In [ ]:
#Translate the amino acid sequences to protein sequences
prot_seq_train = []
aa_seq_train = list(train_seq["#aaseq"])
for i in range(len(aa_seq_train)):
    seq_train = aa_seq_train[i]
    prot_train = translate(seq_train)
    prot_seq_train.append(prot_train)

In [ ]:
train_seq["#pseq"] = prot_seq_train
train_seq = train_seq[["#pseq", "indel_freq", "#aaseq"]]
train_seq = train_seq.drop(["#aaseq"], axis = 1)
train_seq.head()

In [ ]:
prot_seq_test = []
aa_seq_test = list(test_seq["#aaseq"])
for i in range(len(aa_seq_test)):
    seq_test = aa_seq_test[i]
    prot_test = translate(seq_test)
    prot_seq_test.append(prot_test)

In [ ]:
test_seq["#pseq"] = prot_seq_test
test_seq = test_seq[["#pseq", "indel_freq", "#aaseq"]]
test_seq = test_seq.drop(["#aaseq"], axis = 1)
test_seq.head()

In [ ]:
#Drop the sequences with stop codon

train_seq = train_seq[~train_seq['#pseq'].str.contains("\*")]
test_seq = test_seq[~test_seq['#pseq'].str.contains("\*")]

In [ ]:
#Top or bottom 2,000 sequences

train_top = train_seq.sort_values(by = ["indel_freq"], axis = 0, ascending = False)
train_bot = train_seq.sort_values(by = ["indel_freq"], axis = 0, ascending = True)
top_2000 = train_top.head(2000)
bot_2000 = train_bot.head(2000)

In [ ]:
def diff_class(df):
    freq = df["indel_freq"]
    if freq >= 50:
        return 1
    else:
        return 0

In [ ]:
top_2000["class"] = top_2000.apply(diff_class, axis = 1)
bot_2000["class"] = bot_2000.apply(diff_class, axis = 1)

In [ ]:
test_seq["class"] = test_seq.apply(diff_class, axis = 1)

In [ ]:
total_4000 = pd.concat([top_2000, bot_2000], axis = 0)
total_4000.shape

In [ ]:
#Sequences to One-hot encoding vector for CNN
pep_seq = total_4000['#pseq'].tolist()
pep_class = total_4000['class'].tolist()
X_seq = [one_hot_encoder(x) for x in pep_seq]
X_seq = np.transpose(np.asarray(X_seq), (0, 2, 1))
X_seq = X_seq.astype('float32')
Y_class = np.array(pep_class)

In [ ]:
print('Length of data:', X_seq.shape[0])
print('Length of peptides:', X_seq.shape[1])
print('Length of amino acids:', X_seq.shape[2])

In [ ]:
from tensorflow import keras 
from tensorflow.keras import layers
from tensorflow.keras import models
import numpy as np
from sklearn.model_selection import ShuffleSplit
from sklearn.metrics import accuracy_score
from tensorflow.keras import optimizers
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc, precision_recall_curve,average_precision_score, confusion_matrix, f1_score,matthews_corrcoef
from inspect import signature

In [ ]:
#Model path

import os
model_path = "/model_path/models"
model_path = os.path.join(model_path,'Model')
os.makedirs(model_path, exist_ok = True)
print(model_path)

In [ ]:
#Hyperparameter

BATCH_SIZE = 8
PEP_length = 10
AA_type  = 20
CLASS_NAMES = ['enriched','not-enriched']
params = [['CONV', 400, 3, 1],
          ['DROP', 0.5],
          ['POOL', 2, 1],
          ['FLAT'],
          ['DENSE', 50]]
activation_func = 'relu'
regularizer_params = None

In [ ]:
model_version = 1
dict_history = {}
steps_per_epoch = 450
Epochs = 50

In [ ]:
import numpy as np
from sklearn.model_selection import ShuffleSplit
from sklearn.metrics import accuracy_score
from tensorflow.keras import optimizers
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc, precision_recall_curve,average_precision_score, confusion_matrix, f1_score,matthews_corrcoef
from inspect import signature
split_data = ShuffleSplit(n_splits = 10, train_size = None, test_size = 0.1, random_state=1)
acc_per_fold = []
loss_per_fold = []
val_acc_per_fold = []
val_loss_per_fold = []
roc_per_fold = []
roc_fpr = []
roc_tpr = []
precision_fold = []
recall_fold = []
ave_precision_fold = []
n_iter =0
fold_num = []

for train_idx, test_idx in split_data.split(X_seq, Y_class):
    X_train = np.array(X_seq[train_idx])
    X_test = np.array(X_seq[test_idx])
    y_train = np.array(Y_class[train_idx])
    y_test = np.array(Y_class[test_idx])
    model = models.Sequential()
    model.add(layers.Conv1D(filters = params[0][1],kernel_size = params[0][2],strides = params[0][3],activation=activation_func, input_shape=(PEP_length,AA_type),kernel_regularizer=regularizer_params, bias_regularizer=regularizer_params,padding='same'))
    model.add(layers.Dropout(rate = params[1][1])) #add noise
    model.add(layers.MaxPool1D(pool_size = params[2][1], strides = params[2][2]))
    model.add(layers.Flatten())
    model.add(layers.Dense(params[4][1], activation=activation_func, kernel_regularizer=regularizer_params,bias_regularizer=regularizer_params))
    model.add(layers.Dense(1, activation='sigmoid')) #'softmax' for N class
    model.compile(loss='binary_crossentropy', optimizer=optimizers.Adam(learning_rate=0.000075), metrics=['acc'] )
    dict_history[model_version] = model.fit(x=X_train,y=y_train,shuffle=True, steps_per_epoch=steps_per_epoch, epochs=Epochs, batch_size = BATCH_SIZE, validation_data=(X_test, y_test), verbose=2)
    history = dict_history[model_version]
    acc = history.history['acc']
    val_acc = history.history['val_acc']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    train_size = X_train.shape[0]
    test_size = X_test.shape[0]
    scores = model.evaluate(X_test, y_test, verbose=0)
    print(f'Score for fold {n_iter+1}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%\n')
    acc_per_fold.append(acc)
    loss_per_fold.append(loss)
    val_acc_per_fold.append(val_acc)
    val_loss_per_fold.append(val_loss)
    arr_acc = np.array(acc_per_fold)
    arr_loss = np.array(loss_per_fold)
    arr_val_acc = np.array(val_acc_per_fold)
    arr_val_loss = np.array(val_loss_per_fold)
    epochs = range(1, len(acc) + 1)
    pred = model.predict(x=X_test)
    fpr, tpr, thresholds = roc_curve(y_test, pred)
    roc_fpr.append(fpr)
    roc_tpr.append(tpr)
    roc_auc = auc(fpr, tpr)
    roc_per_fold.append(roc_auc)
    precision, recall, thresholds = precision_recall_curve(y_test, pred)
    average_precision = average_precision_score(y_test, pred)
    precision_fold.append(precision)
    recall_fold.append(recall)
    ave_precision_fold.append(average_precision)
    globals()['model_fname_{}'.format(n_iter+1)] = f'BBBphagedisplay_TEST_{n_iter+1}_{model_version:03}.h5'
    model_spath = os.path.join(model_path,f'model_fname_{n_iter+1}')
    model.save(model_spath)
    fold_num.append(n_iter+1)
    n_iter += 1
    epochs = range(1, len(acc) + 1)
    fig = plt.figure(figsize=(10.5,3.5))
    plt.plot(epochs, acc, 'bo', label='Training acc')
    plt.plot(epochs, val_acc, 'r', alpha=0.7, label='Validation acc')
    plt.title(f'Training and validation accuracy of fold {n_iter+1}')
    plt.legend()
    fig = plt.figure(figsize=(10.5,3.5))
    plt.plot(epochs, loss, 'bo', label='Training loss')
    plt.plot(epochs, val_loss, 'r', alpha=0.7, label='Validation loss')
    plt.title(f'Training and validation loss of fold {n_iter+1}')
    plt.legend()
    plt.show()
    pred = model.predict(x=X_test)
    fpr, tpr, thresholds = roc_curve(y_test, pred)
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, color='darkorange',
             lw=2, label='ROC curve (area - %0.2f)' % roc_auc)
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.legend(loc='lower right')
    plt.show()
    plt.cla()
    plt.clf()
    precision, recall, thresholds = precision_recall_curve(y_test, pred)
    average_precision = average_precision_score(y_test, pred)
    step_kwargs = ({'step': 'post'}
                   if 'step' in signature(plt.fill_between).parameters
                   else {})
    plt.step(recall, precision, color='navy', alpha=0.2, where='post', label='Avg. Precision: {0:0.2f}'.format(average_precision))
    plt.fill_between(recall, precision, alpha=0.2, color='navy', **step_kwargs)
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.legend(loc='lower right')
    plt.title(f'Precision recall curve and ROC curve of fold {n_iter+1}')
    plt.show()
    plt.cla()
    plt.clf()

In [ ]:
#Model save
model_fname = f'Cas9Predictor_TEST_{model_version:03}.cv.h5'
model_spath = os.path.join(model_path, model_fname)
print(model_spath)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from cycler import cycler
import matplotlib.lines as mlines
num_plots = 10
n_iter = 0

plt.rcParams["font.size"] = 16
fig, ax = plt.subplots(figsize=(10.5,5.5))
ax.set_prop_cycle('color',[plt.cm.jet(i) for i in np.linspace(0, 1, num_plots)])
b_o = mlines.Line2D([], [], color='black', marker='o', linestyle='None', markersize=8)
b_line = mlines.Line2D([], [], color='black', linestyle='solid')
for i in range(n_iter, num_plots):
    plt.plot(epochs, arr_acc[i], 'o', markersize = 5.5, alpha= 0.8)
    i += 1
for i in range(n_iter, num_plots):
    plt.plot(epochs, arr_val_acc[i], alpha= 0.8, label= f'Fold {i+1}')
    i += 1
legend = plt.legend(loc='upper right', bbox_to_anchor=(1.15, 1), frameon = True, fontsize = 11)
art_legend = plt.gca().add_artist(legend)
legend_2 = plt.legend(handles = [b_o, b_line], labels = ['Accuracy', 'Validation accuracy'], loc='lower right', frameon = True, fontsize = 11)
art_legend_2 = plt.gca().add_artist(legend_2)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
num_plots = 10
n_iter = 0

fig, ax = plt.subplots(figsize=(10.5,5.5))
ax.set_prop_cycle('color',[plt.cm.jet(i) for i in np.linspace(0, 1, num_plots)])
b_o = mlines.Line2D([], [], color='black', marker='o', linestyle='None', markersize=8)
b_line = mlines.Line2D([], [], color='black', linestyle='solid')
for i in range(n_iter, num_plots):
    plt.plot(epochs, arr_loss[i], 'o', markersize = 5.5, alpha= 0.8)
    i += 1
for i in range(n_iter, num_plots):
    plt.plot(epochs, arr_val_loss[i], alpha= 0.8, label= f'Fold {i+1}')
    i += 1
legend = plt.legend(loc='upper right', bbox_to_anchor=(1.15, 1), frameon = True, fontsize = 11)
art_legend = plt.gca().add_artist(legend)
legend_2 = plt.legend(handles = [b_o, b_line], labels = ['Loss', 'Validation loss'], loc='lower right', frameon = True, fontsize = 11)
art_legend_2 = plt.gca().add_artist(legend_2)

In [ ]:
num_plots = 10
n_iter = 0

fig, ax = plt.subplots(figsize=(8.5, 8.5))
ax.set_prop_cycle('color',[plt.cm.jet(i) for i in np.linspace(0, 1, num_plots)])
plt.plot([0, 1], [0, 1], color='navy', linewidth=1, linestyle='--')
for i in range(n_iter, num_plots):
    plt.plot(roc_fpr[i], roc_tpr[i], linewidth=1, label=f'ROC curve (area - %0.2f) of fold {i+1}' % roc_per_fold[i])
    i += 1
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate', fontsize = 20)
plt.ylabel('True Positive Rate', fontsize = 20)
plt.xticks(fontsize = 20)
plt.yticks(fontsize = 20)
legend = plt.legend(loc='upper right', bbox_to_anchor=(1.6, 1), frameon = True, fontsize = 13)
art_legend = plt.gca().add_artist(legend)
plt.show()
plt.cla()
plt.clf()

In [ ]:
num_plots = 10
n_iter = 0

fig, ax = plt.subplots(figsize=(8.5, 8.5))
ax.set_prop_cycle('color',[plt.cm.jet(i) for i in np.linspace(0, 1, num_plots)])
for i in range(n_iter, num_plots):
    step_kwargs = ({'step': 'post'}
                   if 'step' in signature(plt.fill_between).parameters
                   else {})
    plt.step(recall_fold[i], precision_fold[i], color='navy', alpha=0.1, where='post', label='Avg. Precision of each fold : {0:0.2f}'.format(ave_precision_fold[i]))
    plt.fill_between(recall_fold[i], precision_fold[i], alpha=0.05, color='navy', **step_kwargs)
    i += 1
plt.xlabel('Recall', fontsize = 20)
plt.ylabel('Precision', fontsize = 20)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xticks(fontsize = 20)
plt.yticks(fontsize = 20)
legend = plt.legend(loc='upper right', bbox_to_anchor=(1.6, 1), frameon = False, fontsize = 13)
art_legend = plt.gca().add_artist(legend)
plt.title(f'Precision-Recall curve of {i} folds', fontsize = 20)
plt.show()
plt.cla()
plt.clf()

In [ ]:
pred = model.predict(x=X_test)

In [ ]:
#Test the model
select_test = test_seq.sort_values(by = ["indel_freq"], axis = 0, ascending = False)
select_test_top = test_seq.head(100)
select_test_bot = test_seq.tail(100)

In [ ]:
test_200 = pd.concat([select_test_top, select_test_bot], axis = 0)
test_200.shape

In [ ]:
val_pep = test_200['#pseq'].tolist()
val_class = test_200['class'].tolist()
val_encode = [one_hot_encoder(x) for x in val_pep]
val_encode = np.transpose(np.asarray(val_encode), (0, 2, 1))

In [ ]:
model_fit = keras.models.load_model("/model_path/models/Model_num/Cas9Predictor_TEST_{model_version:03}.cv.h5")
val_score = model_fit.predict(val_encode)
val_score_flat = val_score.flatten()

In [ ]:
model_fit.fit(val_encode, val_score_flat)

In [ ]:
from sklearn.metrics import roc_curve, auc, precision_recall_curve,average_precision_score, confusion_matrix, f1_score,matthews_corrcoef
from inspect import signature

plt.rcParams["figure.figsize"] = (6, 6) 
plt.rcParams["font.size"] = 16
fpr, tpr, thresholds = roc_curve(val_class, val_score)
roc_auc = auc(fpr, tpr)
plt.plot(fpr, tpr, color='darkorange',
         lw=2, label='ROC curve (area = %0.3f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1])
plt.ylim([0.0, 1])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend(loc='lower right')
plt.show()
plt.cla()
plt.clf()

precision, recall, thresholds = precision_recall_curve(
    val_class, val_score
)
average_precision = average_precision_score(val_class, val_score)

step_kwargs = ({'step': 'post'}
               if 'step' in signature(plt.fill_between).parameters
               else {})
plt.step(recall, precision, color='navy', alpha=0.2, where='post',
         label='Avg. Precision : {0:0.3f}'.format(average_precision))
plt.fill_between(recall, precision, alpha=0.2, color='navy', **step_kwargs)
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.legend(loc='lower right')
plt.show()
plt.cla()
plt.clf()

In [ ]:
score_test = pd.DataFrame(val_score*100, columns = ["pred_score"])

In [ ]:
test_200 = test_200.reset_index(drop=True)
test_table = pd.concat((test_200, score_test), axis = 1)
test_table.to_csv("/file_location/Result_of_test_data.csv", sep = "\t")